In [9]:
import re

def parse_sos_file(sos_file_path):
    koordsys = None
    coordinates = []
    inside_coordinates = False

    with open(sos_file_path, 'r', encoding='ISO8859-10') as f:
        for line in f:
            line = line.strip()
            
            # Identify the KOORDSYS line
            if line.startswith('...KOORDSYS'):
                parts = line.split()
                if len(parts) >= 2:
                    koordsys = int(parts[1])

            # Check if we've reached the ..NØ section
            if line.startswith('..NØ'):
                inside_coordinates = True
                continue
            
            # If inside the NØ section, extract the coordinate pairs
            if inside_coordinates:
                if re.match(r'^\d+\s+\d+$', line):
                    x, y = line.split()
                    coordinates.append((float(x), float(y)))
                else:
                    # If we hit a line that doesn't match coordinates, break out
                    break

    return koordsys, coordinates

# Example usage:
sos_path = "../datasets/aalesund/1504200/200.sos"
koordsys, coords = parse_sos_file(sos_path)
print("KOORDSYS:", koordsys)
print("Extracted Coordinates:", coords)


KOORDSYS: 110
Extracted Coordinates: [(496779.0, 13727.0), (497712.0, 15418.0)]


In [ ]:
import os
from PIL import Image
import ctypes
import json

# Ensure that coords and koordsys are available from Cell 1

if len(coords) != 2:
    raise ValueError("Expected exactly two coordinate pairs (lower-left and upper-right corners).")

(x_min, y_min), (x_max, y_max) = coords

# Sort coordinates to get proper bounding box
x_min, x_max = sorted([x_min, x_max])
y_min, y_max = sorted([y_min, y_max])

# If koordsys is 110 (Ålesund), transform to EUREF89-32N using the DLL
if koordsys == 110:
    # Load and initialize the transformation
    dll_path = "skt2_1507-1504_1.dll"
    href_file = "HREF2018B_NN2000_EUREF89.bin"
    if not os.path.exists(dll_path):
        raise FileNotFoundError("DLL file not found.")
    if not os.path.exists(href_file):
        raise FileNotFoundError("HREF file not found.")
    
    trans_dll = ctypes.CDLL(dll_path)

    _InitSkTrans = trans_dll._InitSkTrans
    _InitSkTrans.argtypes = [ctypes.c_char_p, ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short)]
    _InitSkTrans.restype = None

    _GeoTrans = trans_dll._GeoTrans
    _GeoTrans.argtypes = [
        ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short),
        ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double),
        ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short),
        ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double),
        ctypes.POINTER(ctypes.c_short)
    ]
    _GeoTrans.restype = None

    href = href_file.encode('utf-8')
    sLen = ctypes.c_short(len(href))
    sErr = ctypes.c_short(0)
    _InitSkTrans(href, ctypes.byref(sLen), ctypes.byref(sErr))
    if sErr.value != 0:
        raise RuntimeError("Failed to init transformation")

    # Ålesund-lokal -> EUREF89-32N
    # According to given info:
    # slSys1=4; slZone1=1 for Ålesund-lokal
    # slSys2=7; slZone2=32 for EUREF89-32N
    slSys1 = ctypes.c_short(4)
    slZone1 = ctypes.c_short(1)
    slSys2 = ctypes.c_short(7)
    slZone2 = ctypes.c_short(32)

    def transform_aalesund_to_euref(x, y):
        x1 = ctypes.c_double(x)
        y1 = ctypes.c_double(y)
        z1 = ctypes.c_double(0.0)
        x2 = ctypes.c_double(0.0)
        y2 = ctypes.c_double(0.0)
        z2 = ctypes.c_double(0.0)
        slErr = ctypes.c_short(0)

        _GeoTrans(
            ctypes.byref(slSys1), ctypes.byref(slZone1),
            ctypes.byref(x1), ctypes.byref(y1), ctypes.byref(z1),
            ctypes.byref(slSys2), ctypes.byref(slZone2),
            ctypes.byref(x2), ctypes.byref(y2), ctypes.byref(z2),
            ctypes.byref(slErr)
        )

        if slErr.value != 0:
            raise RuntimeError("Coordinate transformation failed.")
        return x2.value, y2.value

    # Transform both corners
    (x_min, y_min) = transform_aalesund_to_euref(x_min, y_min)
    (x_max, y_max) = transform_aalesund_to_euref(x_max, y_max)
    print("Coordinates transformed from Ålesund 110 to EUREF89-32N")

else:
    print("Coordinate system is not Ålesund 110. Using coordinates as is.")

image_path = "dataset/aalesund/FOKUS/1504200/200.jpg"
img = Image.open(image_path)
width, height = img.size

pixel_size_x = (x_max - x_min) / width
pixel_size_y = (y_min - y_max) / height

top_left_x = x_min + (pixel_size_x / 2.0)
top_left_y = y_max - (abs(pixel_size_y) / 2.0)

world_file_path = os.path.splitext(image_path)[0] + ".jgw"
with open(world_file_path, 'w') as wf:
    wf.write(f"{pixel_size_x}\n")
    wf.write("0.0\n")
    wf.write("0.0\n")
    wf.write(f"{pixel_size_y}\n")
    wf.write(f"{top_left_x}\n")
    wf.write(f"{top_left_y}\n")

print(f"World file created: {world_file_path}")
print("Image georeferencing parameters have been written.")


FileNotFoundError: Could not find module 'skt2_1507-1504_1.dll' (or one of its dependencies). Try using the full path with constructor syntax.

: 